## 2D images
### Apply provided masks to Pix3D images

In [1]:
import cv2
import numpy as np
import os
from pathlib import Path

pix3d_path = "/home/davide/Desktop/Dissertation/Data/Pix3D/%s/chair/"
imgs_path = pix3d_path % "img"
masks_path = pix3d_path % "mask"

files_pix3d = os.listdir(imgs_path)
files_pix3d = [file.split(".")[0] for file in files_pix3d]

for i in range(len(files_pix3d)):

    img_file = os.path.join(imgs_path, files_pix3d[i]+".jpg")
    mask_file = os.path.join(masks_path, files_pix3d[i]+".png")
    out_file = os.path.join(imgs_path, files_pix3d[i]+".png")
    
    if Path(img_file).is_file() and Path(mask_file).is_file():
        
        img = cv2.imread(img_file)
        mask = cv2.imread(mask_file,0)
        # res = cv2.bitwise_and(img,img,mask = mask)

        transparent = np.zeros((img.shape[0], img.shape[1], 4), dtype=np.uint8)
        transparent[:,:,0:3] = img
        transparent[:, :, 3] = mask

        cv2.imwrite(out_file,transparent)

    print(f"{i+1}/{len(files_pix3d)}")

## Voxels
### Convert OBJ to Binvox

In [ ]:
from pathlib import Path
import numpy as np
import binvox_rw
import os

N_VOX = 32

chair_dir = "/home/davide/Desktop/Dissertation/Data/Pix3D/model/chair"
model_dir = os.path.join(chair_dir, "%s/model.obj")
binvox_dir = os.path.join(chair_dir, "%s")

models_Pix3D = os.listdir(chair_dir)

for i in range(len(models_Pix3D)):
    model_path = model_dir % models_Pix3D[i]
    
    binvox_path = binvox_dir % models_Pix3D[i]
    binvox_path = os.path.join(binvox_path, "model.binvox")

    if not Path(binvox_path).is_file() and Path(model_path).is_file():

        print(f"====== {i+1:4}/{len(models_Pix3D)} {binvox_path} ======")
        # run = f"./binvox {model_path} -d {str(N_VOX)} -e -cb -rotx -rotx -rotx -rotz"

        # Combination for Pix3D
        run = f"./binvox {model_path} -d {str(N_VOX)} -e -cb -rotz -rotx -rotx -rotz -rotz"
        !{run}        

        if Path(binvox_path).is_file():
            with open(binvox_path, 'rb') as file:
                v = binvox_rw.read_as_3d_array(file)

            v.data = np.transpose(v.data, (2, 0, 1))
            with open(binvox_path, 'wb') as file:
                binvox_rw.write(v, file)

### Number models in dataset

In [1]:
import json
import os
from pathlib import Path


json_path = "/home/davide/Desktop/Dissertation/Data/Pix3D/Pix3D.json"

annotations = None
with open(json_path, encoding='utf-8') as file:
    annotations = json.loads(file.read())

json_models = []

for i in range(len(annotations)):
    model_name_parts = annotations[i]['voxel'].split('/')
    model_name = model_name_parts[2]
    
    if model_name not in json_models:
        json_models.append(model_name)
        # print(model_name)

print(f"Total unique model names in JSON file {len(json_models)}")

chair_path = "/home/davide/Desktop/Dissertation/Data/Pix3D/model/chair"
file_binvox_path = os.path.join(chair_path, "%s/model.binvox")

models_Pix3D = os.listdir(chair_path)

models_test = []
binvox_counter = 0

for model in models_Pix3D:
    if model in json_models:
        models_test.append(model)
        # print(model)
        
        binvox_path = file_binvox_path % model
        if Path(binvox_path).is_file():
            binvox_counter = binvox_counter + 1


print(f"Number JSON names available in Pix3D dataset: {len(models_test)}")
print(f"Of which Binvox: {binvox_counter}")

Total unique model names in JSON file 374
Number JSON names available in Pix3D dataset: 221
Of which Binvox: 216
